In [1]:
import org.apache.spark.sql.SparkSession
val sc = SparkSession.builder().getOrCreate()
sc.version

sc = org.apache.spark.sql.SparkSession@2ede49c7


2.4.3

In [5]:
// Chapter 4.
// 덧셈 연산
val df = sc.range(500).toDF("number")
df.select(df.col("number") + 10)

df = [number: bigint]


[(number + 10): bigint]

In [6]:
// 로우
sc.range(2).toDF().collect()

Array([0], [1])

In [8]:
// 스파크 데이터 타입
import org.apache.spark.sql.types._
val b = ByteType

b = ByteType


ByteType

In [17]:
// Chapter 5.
// 스키마 조회
val df = sc.read.format("json").load("./2015-summary.json")
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



df = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [12]:
// structType으로 조회
sc.read.format("json").load("./2015-summary.json").schema

StructType(StructField(DEST_COUNTRY_NAME,StringType,true), StructField(ORIGIN_COUNTRY_NAME,StringType,true), StructField(count,LongType,true))

In [15]:
// 직접 만든 스키마 적용
import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType}
import org.apache.spark.sql.types.Metadata

val myManualSchema = StructType(Array (
    StructField("Dest_Country_Name", StringType, true),
    StructField("Origin_Country_Name", StringType, true),
    StructField("count", LongType, false)))

val df = sc.read.format("json").schema(myManualSchema).load("./2015-summary.json")

myManualSchema = StructType(StructField(Dest_Country_Name,StringType,true), StructField(Origin_Country_Name,StringType,true), StructField(count,LongType,false))
df = [Dest_Country_Name: string, Origin_Country_Name: string ... 1 more field]


[Dest_Country_Name: string, Origin_Country_Name: string ... 1 more field]

In [16]:
// 컬럼
import org.apache.spark.sql.functions.{col, column}
col("someColumnName")
column("someColumnName")

someColumnName

In [21]:
df.col("count")

count

In [22]:
df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [24]:
// 표현식 컬럼 표현
import org.apache.spark.sql.functions.expr

expr("(((count + 5) * 200) - 6) < 100")

((((count + 5) * 200) - 6) < 100)

In [25]:
// dataframe의 column 접근
df.columns

Array(DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count)

In [27]:
// row 생성
import org.apache.spark.sql.Row

val myRow = Row("Hello", null, 1, false)
myRow(0)

myRow = [Hello,null,1,false]


Hello

In [28]:
// Row 접근 (스칼라는 명시적으로 데이터 타입을 지정해야함.)
myRow(0).asInstanceOf[String]

Hello

In [29]:
// SQL 쿼리 실행을 위함
df.createOrReplaceTempView("dfTable")

In [34]:
// select, selectExpr
df.select("DEST_COUNTRY_NAME").show(2)
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

df.select(
    df.col("DEST_COUNTRY_NAME") ,
    col("DEST_COUNTRY_NAME") ,
    column("DEST_COUNTRY_NAME") ,
    $"DEST_COUNTRY_NAME" ,
    expr("DEST_COUNTRY_NAME"))
.show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows

+-----------------+-----------------+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|    United States|    United States|
|    United States|    United States|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+-----------------+-----------------+
only showing top 2 rows



In [35]:
// expr은 유연한 참조 방법임.
df.select(expr("DEST_COUNTRY_NAME AS destination")).show(2)

+-------------+
|  destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows



In [36]:
// select, expr를 합친것 selectExpr
df.selectExpr("DEST_COUNTRY_NAME as newColumnName", "DEST_COUNTRY_NAME").show(2)

+-------------+-----------------+
|newColumnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



In [37]:
// DEST, ORIGIN이 같은지 보고 결과를 새로운 컬럼에 저장. *은 select에서는 사용할 수 없음.
df.selectExpr(
    "*" ,
    "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry"
).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [38]:
// 평균 함수나 count, distinct 등 사용 가능.
df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(2)

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



In [39]:
import org.apache.spark.sql.functions.lit

// One이라는 컬럼에 1값을 명시적으로 채울 수 있음 (컬럼 추가)
df.select(expr("*"), lit(1).as("One")).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



In [40]:
// 다른 방법으로도 컬럼을 추가할 수 있음
df.withColumn("numberOne", lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [41]:
// withColumn은 lit()보다 더 복잡한 로직을 넣을 수 있다.
// lit은 그냥 값 채우기 용.
df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME")).show(5)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
|    United States|            Ireland|  344|        false|
|            Egypt|      United States|   15|        false|
|    United States|              India|   62|        false|
+-----------------+-------------------+-----+-------------+
only showing top 5 rows



In [45]:
// 칼럼명 변경
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").columns
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").show(2)

+-------------+-------------------+-----+
|         dest|ORIGIN_COUNTRY_NAME|count|
+-------------+-------------------+-----+
|United States|            Romania|   15|
|United States|            Croatia|    1|
+-------------+-------------------+-----+
only showing top 2 rows



In [47]:
import org.apache.spark.sql.functions.expr

// withColumn은 공백이나 하이픈(-) 같은 문자에 이스케이프 문자가 필요 없음. 

df.withColumn(
    "This Long Column-Name",
    expr("ORIGIN_COUNTRY_NAME")
).show(2)

+-----------------+-------------------+-----+---------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|This Long Column-Name|
+-----------------+-------------------+-----+---------------------+
|    United States|            Romania|   15|              Romania|
|    United States|            Croatia|    1|              Croatia|
+-----------------+-------------------+-----+---------------------+
only showing top 2 rows



In [49]:
// withColumn이 아닌 selectExpr에서는 백틱(`)문자로 감싸줘야 함.
df.withColumn(
    "This Long Column-Name",
    expr("ORIGIN_COUNTRY_NAME")
).selectExpr(
    "`This Long Column-Name`",
    "`This Long Column-Name` as `new col`"
).show(2)

+---------------------+-------+
|This Long Column-Name|new col|
+---------------------+-------+
|              Romania|Romania|
|              Croatia|Croatia|
+---------------------+-------+
only showing top 2 rows



lastException: Throwable = null


In [50]:
// 대소문자 구분
set spark.sql.caseSensitive true

Name: Unknown Error
Message: <console>:2: error: ';' expected but '.' found.
set spark.sql.caseSensitive true
         ^

StackTrace: 

In [51]:
// 칼럼 삭제
df.drop("ORIGIN_COUNTRY_NAME").columns

Array(DEST_COUNTRY_NAME, count)

In [53]:
// 컬럼의 데이터 타입 변경
df.withColumn("count2", col("count").cast("string")).printSchema()
// SQL -- SELECT *, cast(count as string) as count2

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)
 |-- count2: string (nullable = true)



In [54]:
// 로우 필터링
df.filter(col("count") < 2).show(2) // 프로그래밍 방식
df.where("count < 2").show(2) // SQL 방식

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [55]:
// 고유 로우 얻기 distinct() 함수
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()

256

In [56]:
// 무작위 샘플 추출 (복원 추출, 비복원 추출 모두 제공)
val seed = 5
val withReplacement = false
val fraction = 0.5
df.sample(withReplacement, fraction, seed).count()

seed = 5
withReplacement = false
fraction = 0.5


126

In [57]:
// 임의 분할
// 주로 머신러닝에서 사용
val dataFrames = df.randomSplit(Array(0.25, 0.75), seed)
dataFrames(0).count > dataFrames(1).count()

dataFrames = Array([DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field], [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field])


false

In [62]:
// 로우 합치기, 추가하기
import org.apache.spark.sql.Row

val schema = df.schema

val newRows = Seq (
    Row("New Country", "Other Country", 5L),
    Row("New Country2", "Other Country2", 1L)
)

val parallelizedRows = sc.sparkContext.parallelize(newRows)
val newDF = sc.createDataFrame(parallelizedRows, schema)
df.union(newDF).where("count = 1").where($"ORIGIN_COUNTRY_NAME" =!= "United States").show(100)
// 추가된 로우를 볼 수 있다.

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|    United States|          Gibraltar|    1|
|    United States|             Cyprus|    1|
|    United States|            Estonia|    1|
|    United States|          Lithuania|    1|
|    United States|           Bulgaria|    1|
|    United States|            Georgia|    1|
|    United States|            Bahrain|    1|
|    United States|   Papua New Guinea|    1|
|    United States|         Montenegro|    1|
|    United States|            Namibia|    1|
|     New Country2|     Other Country2|    1|
+-----------------+-------------------+-----+



schema = StructType(StructField(DEST_COUNTRY_NAME,StringType,true), StructField(ORIGIN_COUNTRY_NAME,StringType,true), StructField(count,LongType,true))
newRows = List([New Country,Other Country,5], [New Country2,Other Country2,1])
parallelizedRows = ParallelCollectionRDD[148] at parallelize at <console>:58
newDF = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [63]:
// 정렬하기
df.sort("count").show(5)
df.orderBy("count", "DEST_COUNTRY_NAME")show(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [64]:
// 로우 수 제한하기
df.limit(5).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+



In [70]:
// repartition, coalesce
df.rdd.getNumPartitions
df.repartition(5)
df.rdd.getNumPartitions

1

In [71]:
// 파티션 수와 칼럼을 선택할수도 있음.
df.repartition(5, col("DEST_COUNTRY_NAME"))

[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [72]:
df.repartition(5, col("DEST_COUNTRY_NAME")).coalesce(2)

[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [73]:
// 드라이버로 로우 수집
val collectDF = df.limit(10)
collectDF.take(5)
collectDF.show()
collectDF.show(5, false)
collectDF.collect()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India         

collectDF = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


Array([United States,Romania,15], [United States,Croatia,1], [United States,Ireland,344], [Egypt,United States,15], [United States,India,62], [United States,Singapore,1], [United States,Grenada,62], [Costa Rica,United States,588], [Senegal,United States,40], [Moldova,United States,1])

In [74]:
collectDF.show(5, false)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
+-----------------+-------------------+-----+
only showing top 5 rows

